In [ ]:
import numpy as np
import pandas as pd
import os
from inda_mir.loaders import load_model, load_data_partition
from settings import settings
from tqdm import tqdm

In [ ]:
misd_path = '/mnt/y/CODES/track_classifier/misd'
train_csv_path = os.path.join(misd_path, 'Metadata_Train.csv')
test_csv_path = os.path.join(misd_path, 'Metadata_Test.csv')

In [ ]:
train_metadata = pd.read_csv(os.path.join(misd_path, 'train.csv'))
test_metadata = pd.read_csv(os.path.join(misd_path, 'test.csv'))
train_path = os.path.join(misd_path, 'train')
test_path = os.path.join(misd_path, 'test')

In [ ]:
train_metadata['Class'].value_counts()

Class
Sound_Guitar    700
Sound_Drum      700
Sound_Violin    700
Sound_Piano     529
Name: count, dtype: int64

In [ ]:
import os

def rename_files_to_lowercase(directory):
    for filename in os.listdir(directory):
        if os.path.isfile(os.path.join(directory, filename)):
            # Get the lowercase version of the filename
            lowercase_filename = filename.lower()
            
            # Get the path to the lowercase filename
            lowercase_filepath = os.path.join(directory, lowercase_filename)
            
            # Rename the file to the lowercase version
            os.rename(os.path.join(directory, filename), lowercase_filepath.replace("-", "_"))


# rename_files_to_lowercase(train_path)
# rename_files_to_lowercase(test_path)
# train_metadata['FileName'] = train_metadata['FileName'].apply(lambda x: x.lower().replace("-", "_"))
# test_metadata['FileName'] = test_metadata['FileName'].apply(lambda x: x.lower().replace("-", "_"))

# train_metadata['FileName'] = train_metadata['FileName'].apply(lambda x: x.replace(" ", ""))
# test_metadata['FileName'] = test_metadata['FileName'].apply(lambda x: x.replace(" ", ""))

# train_metadata['path'] = train_metadata['FileName'].apply(lambda x: os.path.join(train_path, x))
# test_metadata['path'] = test_metadata['FileName'].apply(lambda x: os.path.join(test_path, x))

train_metadata['prediction'] = None
test_metadata['prediction'] = None

# train_metadata.to_csv(os.path.join(misd_path, 'train.csv'))
# test_metadata.to_csv(os.path.join(misd_path, 'test.csv'))

In [ ]:
def delete_ogg_files(directory):
    for filename in os.listdir(directory):
        if filename.endswith('.ogg'):
            file_path = os.path.join(directory, filename)
            try:
                os.remove(file_path)
                print(f"Deleted: {filename}")
            except OSError as e:
                print(f"Error deleting {filename}: {e}")

# Specify your directory - exercise caution!
directory = test_path

# Optional: Confirmation before proceeding
confirm = input("Delete all .ogg files in the directory? (yes/no): ")
if confirm.lower() == 'yes' or confirm.lower() == 'y':
    delete_ogg_files(directory)
else:
    print("Deletion cancelled.")

Deletion cancelled.


In [ ]:
train_metadata.head()

,Unnamed: 0,FileName,Class,path,prediction
0,0,1_e1_major00.wav,Sound_Guitar,/mnt/y/CODES/track_classifier/misd/train/1_e1_...,None
1,1,1_e1_major01.wav,Sound_Guitar,/mnt/y/CODES/track_classifier/misd/train/1_e1_...,None
2,2,1_e1_major02.wav,Sound_Guitar,/mnt/y/CODES/track_classifier/misd/train/1_e1_...,None
3,3,1_e1_major03.wav,Sound_Guitar,/mnt/y/CODES/track_classifier/misd/train/1_e1_...,None
4,4,1_e1_major04.wav,Sound_Guitar,/mnt/y/CODES/track_classifier/misd/train/1_e1_...,None


In [ ]:
train_metadata['path'].iloc[0]

'/mnt/y/CODES/track_classifier/misd/train/1_e1_major_00.wav'

In [ ]:
model = load_model(os.path.join('/mnt/y/CODES/track_classifier/output/models/lgbm_retrained_00_46_21_27_03_2024.pkl'))
model

In [ ]:
model.get_n_features()

253

In [ ]:
from inda_mir.modeling.models.classifier import InstrumentClassifier


classifier = InstrumentClassifier(model)

In [ ]:
def get_predictions(df: pd.DataFrame, classifier: InstrumentClassifier, output_path) -> pd.DataFrame:
    
    for i, row in tqdm(df[df['prediction'].isna()].iterrows()):
        pred = classifier.classify(row['path']).instrument
        df.loc[i, 'prediction'] = pred
        df.to_csv(os.path.join(output_path))
    return df

In [ ]:
# %%capture
get_predictions(train_metadata, classifier, os.path.join(misd_path, 'train.csv'))

0it [00:00, ?it/s]2024-04-17 18:06:41,718 - INFO - classifier - Extracting 1_e1_major_00 features...


2024-04-17 18:06:41,808 - INFO - classifier - Classified "1_e1_major_00" as "silence"!
1it [00:00,  8.61it/s]2024-04-17 18:06:41,834 - INFO - classifier - Extracting 1_e1_major_01 features...


/mnt/y/CODES/track_classifier/misd/train/1_e1_major_00.wav
1_e1_major_00 wav
/mnt/y/CODES/track_classifier/misd/train/1_e1_major_01.wav
1_e1_major_01 wav


2024-04-17 18:06:41,925 - INFO - classifier - Classified "1_e1_major_01" as "silence"!
2it [00:00,  8.51it/s]2024-04-17 18:06:41,953 - INFO - classifier - Extracting 1_e1_major_02 features...
2024-04-17 18:06:42,037 - INFO - classifier - Classified "1_e1_major_02" as "silence"!
3it [00:00,  8.70it/s]2024-04-17 18:06:42,066 - INFO - classifier - Extracting 1_e1_major_03 features...
2024-04-17 18:06:42,149 - INFO - classifier - Classified "1_e1_major_03" as "silence"!


/mnt/y/CODES/track_classifier/misd/train/1_e1_major_02.wav
1_e1_major_02 wav
/mnt/y/CODES/track_classifier/misd/train/1_e1_major_03.wav
1_e1_major_03 wav


...

/mnt/y/CODES/track_classifier/misd/train/ar_lick10_fn.wav
ar_lick10_fn wav


2024-04-17 18:06:52,448 - INFO - classifier - Classified "ar_lick10_fn" as "silence"!
36it [00:10,  1.99it/s]2024-04-17 18:06:52,469 - INFO - classifier - Extracting ar_lick10_kn features...


/mnt/y/CODES/track_classifier/misd/train/ar_lick10_kn.wav
ar_lick10_kn wav


36it [00:11,  3.25it/s]


KeyboardInterrupt: 

In [ ]:
test_metadata.prediction.value_counts()

prediction
silence    80
Name: count, dtype: int64